# Prueba técnica lulo bank

### A continuación se mostrarán los resultados y análisis realizados a la data obtenida:
En primer lugar se instalan los requerimientos para la ejecución de este notebook

In [1]:
!pip install -r requirements_notebook.txt

  Using cached MarkupSafe-2.1.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aws-sam-cli 1.53.0 requires MarkupSafe==2.0.1, but you have markupsafe 2.1.1 which is incompatible.


Se realiza la importación de librerías

In [1]:
import json
import pandas as pd
import os
import numpy as np
import multiprocessing as mp
from pandas_profiling import ProfileReport

Se realiza la carga de los archivos JSON que se descargaron previamente, se valida de que tengan extensión JSON

In [2]:
dir = sorted(os.listdir("app/json_response"))
df = pd.DataFrame()
for file in dir:
    if file.endswith(".json"):
        print("loading file: ---->", file)
        df_read = pd.read_json(f"app/json_response/{file}")
        df=pd.concat([df, df_read], ignore_index= True)

loading file: ----> 2020-12-01.json
loading file: ----> 2020-12-02.json
loading file: ----> 2020-12-03.json
loading file: ----> 2020-12-04.json
loading file: ----> 2020-12-05.json
loading file: ----> 2020-12-06.json
loading file: ----> 2020-12-07.json
loading file: ----> 2020-12-08.json
loading file: ----> 2020-12-09.json
loading file: ----> 2020-12-10.json
loading file: ----> 2020-12-11.json
loading file: ----> 2020-12-12.json
loading file: ----> 2020-12-13.json
loading file: ----> 2020-12-14.json
loading file: ----> 2020-12-15.json
loading file: ----> 2020-12-16.json
loading file: ----> 2020-12-17.json
loading file: ----> 2020-12-18.json
loading file: ----> 2020-12-19.json
loading file: ----> 2020-12-20.json
loading file: ----> 2020-12-21.json
loading file: ----> 2020-12-22.json
loading file: ----> 2020-12-23.json
loading file: ----> 2020-12-24.json
loading file: ----> 2020-12-25.json
loading file: ----> 2020-12-26.json
loading file: ----> 2020-12-27.json
loading file: ----> 2020-12-

El primer análisis que se realizó fue ver los campos que se obtuvieron de las peticiones al API, me di cuenta de que algunos campos traían diccionarios los cuales eran innecesarios en este caso, también el campo de resumen traía basura por lo cual realice depuración y ajuste de la data teniendo en cuenta lo mencionado anteriormente

In [3]:
df.head()

,id,url,name,season,number,type,airdate,airtime,airstamp,runtime,rating,image,summary,_links,_embedded
0,1979824,https://www.tvmaze.com/episodes/1979824/sim-fo...,Chanyeol's Episode 16,4,16.0,regular,2020-12-01,06:00,2020-11-30T21:00:00+00:00,16.0,{'average': None},None,<p><b>#ObtainedAConversationalSkill #WeSetUpAT...,{'self': {'href': 'https://api.tvmaze.com/epis...,"{'show': {'id': 41648, 'url': 'https://www.tvm..."
1,1979222,https://www.tvmaze.com/episodes/1979222/kotiki...,Серия 2,1,2.0,regular,2020-12-01,,2020-12-01T00:00:00+00:00,12.0,{'average': None},None,None,{'self': {'href': 'https://api.tvmaze.com/epis...,"{'show': {'id': 52198, 'url': 'https://www.tvm..."
2,2008027,https://www.tvmaze.com/episodes/2008027/lab-s-...,Лолита,2,6.0,regular,2020-12-01,,2020-12-01T00:00:00+00:00,29.0,{'average': None},{'medium': 'https://static.tvmaze.com/uploads/...,None,{'self': {'href': 'https://api.tvmaze.com/epis...,"{'show': {'id': 52933, 'url': 'https://www.tvm..."
3,1964565,https://www.tvmaze.com/episodes/1964565/core-s...,Episode 9,1,9.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,24.0,{'average': None},None,None,{'self': {'href': 'https://api.tvmaze.com/epis...,"{'show': {'id': 51336, 'url': 'https://www.tvm..."
4,2052503,https://www.tvmaze.com/episodes/2052503/wu-she...,Episode 80,1,80.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,8.0,{'average': None},None,None,{'self': {'href': 'https://api.tvmaze.com/epis...,"{'show': {'id': 54033, 'url': 'https://www.tvm..."


In [4]:
def cleaning(df):
    df['summary'] = df['summary'].str.replace('<b>','').str.replace('</p>','').str.replace('\xa0','').str.replace('<p>','').str.replace('<br />','').str.replace('</b>','').str.replace('<i>','').str.replace('</i>','')
    df[['self']] = df['_links'].apply(pd.Series)
    df[['link']] = df['self'].apply(pd.Series)
    df[['average_rating']] = df['rating'].apply(pd.Series)
    df[['embedded_show']] = df['_embedded'].apply(pd.Series)
    df = df.drop(['self','rating','_links','_embedded'], axis=1)
    return df 

def parallelize_dataframe(df, func, n_cores):
    df_split = np.array_split(df, n_cores)
    pool = mp.Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

n_cores = mp.cpu_count() 
df = parallelize_dataframe(df, cleaning, n_cores)

runtime=[]
averageRuntime=[]
genres=[]
genres_extend=[]
officialSite=[]
for i in df.index:
    runtime.append(df.embedded_show[i]['runtime'])
    averageRuntime.append(df.embedded_show[i]['averageRuntime'])
    genres.append(df.embedded_show[i]['genres'])
    officialSite.append(df.embedded_show[i]['officialSite'])

runtime = [x for x in runtime if str(x) != 'None']
averageRuntime = [x for x in averageRuntime if str(x) != 'None']
officialSite=[x for x in officialSite if str(x) != 'None']

for e in genres:
    genres_extend.extend(e)

Como resultado obtuve un dataset más limpio y ordenado

In [5]:
df.head(3)

,id,url,name,season,number,type,airdate,airtime,airstamp,runtime,image,summary,link,average_rating,embedded_show
0,1979824,https://www.tvmaze.com/episodes/1979824/sim-fo...,Chanyeol's Episode 16,4,16.0,regular,2020-12-01,06:00,2020-11-30T21:00:00+00:00,16.0,None,#ObtainedAConversationalSkill #WeSetUpATent♥ #...,https://api.tvmaze.com/episodes/1979824,NaN,"{'id': 41648, 'url': 'https://www.tvmaze.com/s..."
1,1979222,https://www.tvmaze.com/episodes/1979222/kotiki...,Серия 2,1,2.0,regular,2020-12-01,,2020-12-01T00:00:00+00:00,12.0,None,None,https://api.tvmaze.com/episodes/1979222,NaN,"{'id': 52198, 'url': 'https://www.tvmaze.com/s..."
2,2008027,https://www.tvmaze.com/episodes/2008027/lab-s-...,Лолита,2,6.0,regular,2020-12-01,,2020-12-01T00:00:00+00:00,29.0,{'medium': 'https://static.tvmaze.com/uploads/...,None,https://api.tvmaze.com/episodes/2008027,NaN,"{'id': 52933, 'url': 'https://www.tvmaze.com/s..."


Se realizaron operaciones de agregación para obtener el número shows de TV por género, media de averageRuntime, media de Runtime y los dominios (web) del sitio oficial de los shows, cabe mencionar que para que la media no se viera afectada no se tuvieron en cuenta los registros vacíos

In [6]:
print("Número shows de TV por género para el mes de diciembre del 2020:")
unique_genres = sorted(list(set(genres_extend)))
for i in unique_genres:
    count = genres_extend.count(i)
    print(f"{i}: {count}")

Número shows de TV por género para el mes de diciembre del 2020:
Action: 215
Adult: 11
Adventure: 186
Anime: 130
Children: 184
Comedy: 750
Crime: 283
DIY: 4
Drama: 1106
Family: 107
Fantasy: 219
Food: 52
History: 144
Horror: 71
Legal: 5
Medical: 10
Music: 113
Mystery: 235
Nature: 22
Romance: 579
Science-Fiction: 93
Sports: 64
Supernatural: 69
Thriller: 221
Travel: 29
War: 30


In [7]:
mean_averageRuntime = f"{(np.mean(averageRuntime)):.2f}"
print(f"Media de averageRuntime = {mean_averageRuntime}")

mean_runtime = f"{(np.mean(runtime)):.2f}"
print(f"Media de Runtime = {mean_runtime}")

Media de averageRuntime = 36.96
Media de Runtime = 38.81


In [8]:
print("Dominios (web) del sitio oficial de los shows: ")
unique_domains = sorted(list(set(officialSite)))
for domain in unique_domains:
    print(f"---> {domain}")

Dominios (web) del sitio oficial de los shows: 
---> http://1tv.com.ua/programs/maikl_shcur
---> http://epic-media.ru/project/kotiki
---> http://g1.globo.com/fantastico/
---> http://gtst.nl/#!/
---> http://heyqween.tv/shows/hey-qween/episodes/
---> http://idolish7.com/aninana/
---> http://kurzgesagt.org/
---> http://m-78.jp/galaxy-fight/tac/
---> http://network.wwe.com/shows/original/wwe-24
---> http://network.wwe.com/shows/original/wwe-untold
---> http://seasonvar.ru/serial-11488-Po_sezonu_Videodajdzhest_Seasonvar.html
---> http://seasonvar.ru/serial-12772-TOP-10_po_versii_Seasonvarru-1-season.html
---> http://simonscat.com
---> http://stopgame.ru/razbor/new
---> http://talkingtomandfriends.com/animated-series/
---> http://thegameawards.com
---> http://tv3.ru/project/slepaya
---> http://vmesteproject.ru/intervue/
---> http://weibo.com/u/6516179447
---> http://www.atopthefourthwall.com
---> http://www.bbc.co.uk/programmes/b0959g26
---> http://www.bbc.co.uk/programmes/p05j1jkp
---> http

Se realiza una visualización del número de nulos que existe en cada columna y se realiza un análisis descriptivo de cada campo

In [9]:
print("Revisión número de nulos por columna:")
df.isna().sum()

Revisión número de nulos por columna:


id                   0
url                  0
name                 0
season               0
number              81
type                 0
airdate              0
airtime              0
airstamp             0
runtime            227
image             2026
summary           2147
link                 0
average_rating    2736
embedded_show        0
dtype: int64

In [10]:
df.describe(include='all')

,id,url,name,season,number,type,airdate,airtime,airstamp,runtime,image,summary,link,average_rating,embedded_show
count,3.037000e+03,3037,3037,3037.000000,2956.000000,3037,3037,3037,3037,2810.000000,1011,890,3037,301.000000,3037
unique,NaN,3037,2090,NaN,NaN,3,31,61,536,NaN,1011,885,3037,NaN,624
top,NaN,https://www.tvmaze.com/episodes/1979824/sim-fo...,Episode 1,NaN,NaN,regular,2020-12-04,,2020-12-18T12:00:00+00:00,NaN,{'medium': 'https://static.tvmaze.com/uploads/...,"On a journey to find love, Chance invites 15 b...",https://api.tvmaze.com/episodes/1979824,NaN,"{'id': 15250, 'url': 'https://www.tvmaze.com/s..."
freq,NaN,1,65,NaN,NaN,2956,176,2135,102,NaN,1,3,1,NaN,38
mean,2.029997e+06,NaN,NaN,149.383602,23.325101,NaN,NaN,NaN,NaN,37.760854,NaN,NaN,NaN,7.943522,NaN
std,9.197163e+04,NaN,NaN,524.145917,51.795918,NaN,NaN,NaN,NaN,30.120851,NaN,NaN,NaN,1.024792,NaN
min,1.732625e+06,NaN,NaN,1.000000,1.000000,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,3.700000,NaN
25%,1.978095e+06,NaN,NaN,1.000000,4.000000,NaN,NaN,NaN,NaN,19.000000,NaN,NaN,NaN,7.300000,NaN
50%,1.989343e+06,NaN,NaN,1.000000,8.000000,NaN,NaN,NaN,NaN,32.000000,NaN,NaN,NaN,7.900000,NaN
75%,2.036955e+06,NaN,NaN,3.000000,20.000000,NaN,NaN,NaN,NaN,45.000000,NaN,NaN,NaN,8.700000,NaN


Como resultado de las peticiones a la API http://api.tvmaze.com para el mes de diciembre se obtuvieron 3037 registros también se puede inferir que al rededor de la tercera parte de estos registros no trae links de imágenes. Considere ajustar los tipos de datos de las columnas del dataframe para que tengan relación el contenido de las filas con el tipo de dato que contiene. Finalmente se realiza el profiling de la data depurada y se guarda la información en un archivo de extensión parquet

In [11]:
df = df.astype({"image":str,"embedded_show":str})
df['airstamp']= pd.to_datetime(df['airstamp'])

profile = ProfileReport(df,
                        title="Series emitidas en diciembre del 2020",
        dataset={
        "description": "Profiling report generado por Alexander",
        "copyright_holder": "Analytics",
        "copyright_year": "2020",
        "url": " http://api.tvmaze.com ",
    },)
profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
profile.to_file("Profile.html")
df['airdate'] = pd.to_datetime(df['airdate']).dt.date
df['airtime'] = df['airtime'].replace('',np.nan)
df['airtime'] = pd.to_datetime(df['airtime'],format= '%H:%M').dt.time 
df.to_parquet('app/data.parquet', compression='snappy')

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Análisis del profiling realizado
A continuación mencionaré algunos puntos que destaco como conclusiones:
* Se puede inferir que los programas tienen una franja horaria preestablecida, lo que conlleva que en ciertas horas tengan más rating.
* El rating está estrechamente relacionado con el tipo de programa y la franja horaria en la que se emite
* La mayoría de los shows que se emitieron pertenecen a la temporada 1 del mismo
* El 97 porciento de los programas emitidos son de carácter regular
* Tienen preestablecidas fechas y horas de emisión para los programas